In [1]:
import numpy as np
from Dataset import load_dataset, create_set

from ConvolutionalRGB import ConvolutionalRGB
from Activation import Activation
from PoolingRGB import Pooling_maxRGB
from Reshape import Reshape
from Dense import Dense
from Softmax import Softmax
from Network import Network
from Functions import ReLU, ReLU_derivative, tanh, tanh_derivative, mse, mse_derivative, sigmoid, sigmoid_derivative, binary_cross_entropy, binary_cross_entropy_derivative

In [2]:
dataset = load_dataset('./../dataset/new_dataset.csv', './../dataset/images/')

In [6]:
classes = [0, 1]
x_train, y_train, remaining  = create_set(dataset, classes, 150)
x_test, y_test, remaining = create_set(remaining, classes, 50)

print(x_train.shape)
print(y_test.shape)

(300, 144, 256, 3)
(100, 2, 1)


In [ ]:
network = Network([
    ConvolutionalRGB(input_shape=(1, 144, 256), kernel_size=3, depth=8),
    Activation(ReLU, ReLU_derivative),
    ConvolutionalRGB(input_shape=(8, 142, 254), kernel_size=5, depth=8),
    Activation(ReLU, ReLU),
    Pooling_maxRGB(input_shape=(8, 138, 250), kernel_shape=(2, 2), stride=(2, 2)),
    
    ConvolutionalRGB(input_shape=(8, 69, 125), kernel_size=3, depth=4),
    Activation(ReLU, ReLU_derivative),
    ConvolutionalRGB(input_shape=(4, 67, 123), kernel_size=5, depth=4),
    Activation(ReLU, ReLU),
    Reshape(input_shape=(4, 63, 119, 3), output_shape=(4 * 63 * 119 * 3, 1)),

    Dense(4 * 63 * 119 * 3, 32),
    Activation(ReLU, ReLU),
    Dense(32, 2),
    Activation(tanh, tanh_derivative)
    #Softmax()
])

In [ ]:
network.train_stocastic(loss=mse, 
                        loss_derivative=mse_derivative, 
                        X=x_train, 
                        Y=y_train, 
                        epochs=2,
                        batches=32,
                        learning_rate=0.005, 
                        print_debug=True)

In [ ]:
count = 0
for x, y in zip(x_test, y_test):
    prediction = network.predict(x)

    print(f"pred: {np.argmax(prediction)} ({prediction}), \ttrue: {np.argmax(y)}")
    
    if np.argmax(prediction) == np.argmax(y):
        count += 1

print(f'{count}/{len(x_test)} images classified correctly.')